In [1]:
from transformers import AutoModelForSeq2SeqLM
import peft
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, AdaMixConfig, TaskType
import torch
import torch.nn.functional as F 
from datasets import load_dataset
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cuda"
model_name_or_path = "t5-small"
tokenizer_name_or_path = "t5-small"

checkpoint_name = "financial_sentiment_analysis_adamix_v1.pt"
text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 3e-4
num_epochs = 5
batch_size = 8

2023-07-19 21:54:18.365356: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/slurm/23.02/lib:/nfs/hpc/share/balasuri/research_env/lib
2023-07-19 21:54:18.365462: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/slurm/23.02/lib:/nfs/hpc/share/balasuri/research_env/lib
2023-07-19 21:54:18.365486: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
import importlib

importlib.reload(peft)

<module 'peft' from '/nfs/hpc/share/balasuri/peft/src/peft/__init__.py'>

In [3]:
# creating model
peft_config = AdaMixConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, return_two_views=True, adapter_dim=4, num_expert=4)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

In [4]:
print(model)
sum(p.numel() for p in model.parameters() if p.requires_grad)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

60506624

In [5]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

trainable params: 129,216 || all params: 60,635,840 || trainable%: 0.21310169035342794


PeftModelForSeq2SeqLM(
  (base_model): AdaMixModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(in_features=512, out_features=512, bias=False)
                  (k): Linear(in_features=512, out_features=512, bias=False)
                  (v): Linear(in_features=512, out_features=512, bias=False)
                  (o): Linear(in_features=512, out_features=512, bias=False)
                  (relative_attention_bias): Embedding(32, 8)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseActDense(
                  (wi): Linear(in_feat

In [6]:
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Found cached dataset financial_phrasebank (/nfs/stak/users/balasuri/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'Finnish electronics contract manufacturer Scanfil had net sales of EUR 52.2 mn in the first quarter of 2007 , down from EUR 60.1 mn a year before .',
 'label': 0,
 'text_label': 'negative'}

In [7]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=3, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [8]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [12]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    model.inference_mode = False
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        views = model.get_two_view_from_model()
        views = views[6:] # only the decoder views
        views = torch.stack(views)
        p, q = views[:, 0].permute(0, 1, 3, 2), views[:, 1].permute(0, 1, 3, 2)
        dim = p.shape[-1]
        p = p.reshape(-1, dim)
        q = q.reshape(-1, dim)
        l = F.kl_div(F.log_softmax(p, dim=-1, dtype=torch.float32),
                     F.softmax(q.detach(), dim=-1, dtype=torch.float32), reduction='batchmean') +\
            F.kl_div(F.log_softmax(q, dim=-1, dtype=torch.float32),
                     F.softmax(p.detach(), dim=-1, dtype=torch.float32), reduction='batchmean')
        loss = loss + 0.5 * l.mean()
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
#         lr_scheduler.step()
        optimizer.zero_grad()
        
    model.eval()
    model.inference_mode = True
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 53.29it/s]


epoch=0: train_ppl=tensor(3.8474, device='cuda:0') train_epoch_loss=tensor(1.3474, device='cuda:0') eval_ppl=tensor(1.4108, device='cuda:0') eval_epoch_loss=tensor(0.3442, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 55.68it/s]


epoch=1: train_ppl=tensor(1.5666, device='cuda:0') train_epoch_loss=tensor(0.4489, device='cuda:0') eval_ppl=tensor(1.3267, device='cuda:0') eval_epoch_loss=tensor(0.2827, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 53.70it/s]


epoch=2: train_ppl=tensor(1.4526, device='cuda:0') train_epoch_loss=tensor(0.3734, device='cuda:0') eval_ppl=tensor(1.2674, device='cuda:0') eval_epoch_loss=tensor(0.2369, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 57.00it/s]


epoch=3: train_ppl=tensor(1.3561, device='cuda:0') train_epoch_loss=tensor(0.3046, device='cuda:0') eval_ppl=tensor(1.2203, device='cuda:0') eval_epoch_loss=tensor(0.1991, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 55.28it/s]

epoch=4: train_ppl=tensor(1.2758, device='cuda:0') train_epoch_loss=tensor(0.2436, device='cuda:0') eval_ppl=tensor(1.1280, device='cuda:0') eval_epoch_loss=tensor(0.1204, device='cuda:0')


In [13]:
# print accuracy
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    pred = pred.split(' ')[0]
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=93.83259911894272 % on the evaluation dataset
eval_preds[:10]=['neutral', 'negative', 'neutral', 'neutral', 'negative', 'positive', 'neutral', 'neutral', 'neutral', 'positive']
dataset['validation']['text_label'][:10]=['neutral', 'negative', 'neutral', 'neutral', 'negative', 'positive', 'neutral', 'neutral', 'neutral', 'positive']
